In [ ]:
%cd drive/MyDrive/kwik

In [ ]:
!pip install bit

In [ ]:
!python PubSubs.py 0229DFE4805CD534B91A6289EAEDF7EDC6F158EBA3F00D0DD5BE6FB2F7A72E294D 100000000 129 6000.bin cpub

In [ ]:
!python kurang.py

In [ ]:
!python tambah.py

In [ ]:
!python single.py

In [ ]:
!python Pubadd.py 10 120 tes90.txt cpub

In [ ]:
!cat PubSubs.py

In [ ]:
!cat test.txt

In [ ]:
%%writefile PubSubs.py
# python PubSub.py 02DE27F4BBF33E7C4030B32AD4B99CD09B35D130B8F0580F57F2FC2ED77946F51C 100000000 120 tes.bin cpub
# 00000000000000000000000000000001FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF 02DE27F4BBF33E7C4030B32AD4B99CD09B35D130B8F0580F57F2FC2ED77946F51C tes.bin
# 00000000000000000000000000000002A3D70A3D70A3D70A3D70A3D70A3D70A1 02EE88588CC7189EC2CC91901924277DAE22D0F6A5CA07E63FDF017E10140F9EE4 tes1.bin
# 0000000000000000000000000000000213D70A3D70A3D70A3D70A3D70A3D70A1 02BD6C9DDCE8E112124F25E5D8C7AEA764ABD68A3A05952931644932BFD0695B5C tes2.bin
# 00000000000000000000000000000001C28F5C28F5C28F5C28F5C28F5C28F5C1 02627384D7BA7D5B92FC9D11F8D63E5A7ECBF5264D975BE80155A032D9AAA371DB tesa.bin
import bit
import hashlib
import random
import platform
from time import time
import os
import sys
import ctypes
if len(sys.argv) > 6 or len(sys.argv) < 6:
    print('[+] Program Usage.... ')
    print('{} <original_pubkey> <num_of_subkeys> <bit_range> <file_name> <output_mode>\n'.format(sys.argv[0]))
    print('Example to create a text File [upub]:\n{} 02CEB6CBBCDBDF5EF7150682150F4CE2C6F4807B349827DCDBDD1F2EFA885A2630 100000 120 keysub120.txt upub'.format(sys.argv[0]))
    print('Example to create a binary File [xpoint]:\n{} 02CEB6CBBCDBDF5EF7150682150F4CE2C6F4807B349827DCDBDD1F2EFA885A2630 100000 120 keysub120.bin xpoint'.format(sys.argv[0]))
    sys.exit()
pubkey = sys.argv[1]
n = int(sys.argv[2])
nbits = int(sys.argv[3])
out_file = sys.argv[4]  # False if need to write Text File, line by line
out_mode = sys.argv[5]   # upub, cpub, xpoint
if out_mode not in ['upub', 'cpub', 'xpoint']:
    print('Output Mode not specified. [upub or cpub or xpoint]'); sys.exit()
flag_binary_file = False if out_file.split('.')[1] in ['txt','text','asc'] else True
if flag_binary_file is True: print('[+] Detecting output file extension. Output will be written as Binary File')
else: print('[+] Detecting output file extension. Output will be written as Text File')
low = 2**(nbits-1)
high = -1+2**nbits
diff = high - low
flag_write = 'wb' if flag_binary_file else 'w'
out = open(out_file, flag_write)
###############################################################################
if platform.system().lower().startswith('win'):
    dllfile = 'ice_secp256k1.dll'
    if os.path.isfile(dllfile) == True:
        pathdll = os.path.realpath(dllfile)
        ice = ctypes.CDLL(pathdll)
    else:
        print('File {} not found'.format(dllfile))
elif platform.system().lower().startswith('lin'):
    dllfile = 'ice_secp256k1.so'
    if os.path.isfile(dllfile) == True:
        pathdll = os.path.realpath(dllfile)
        ice = ctypes.CDLL(pathdll)
    else:
        print('File {} not found'.format(dllfile))
else:
    print('[-] Unsupported Platform currently for ctypes dll method. Only [Windows and Linux] is working')
    sys.exit()
#ice = ctypes.CDLL('ice_secp256k1.dll')
ice.scalar_multiplication.argtypes = [ctypes.c_char_p, ctypes.c_char_p]            # pvk,ret
ice.point_subtraction.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p] # x1,y1,x2,y2,ret
ice.init_secp256_lib()
###############################################################################
def scalar_multiplication(kk):
    res = (b'\x00') * 65
    pass_int_value = hex(kk)[2:].encode('utf8')
    ice.scalar_multiplication(pass_int_value, res)
    return res
def point_subtraction(pubkey1_bytes, pubkey2_bytes):
    x1 = pubkey1_bytes[1:33]
    y1 = pubkey1_bytes[33:]
    x2 = pubkey2_bytes[1:33]
    y2 = pubkey2_bytes[33:]
    res = (b'\x00') * 65
    ice.point_subtraction(x1, y1, x2, y2, res)
    return res
def new_pos(full_bytes):
    pos = hashlib.sha256(full_bytes).digest()
    return pos
def fixrange(full_bytes):
    t = low + int(full_bytes.hex(), 16) % diff
    return t
def pub2upub(pub_hex):
 x = int(pub_hex[2:66],16)
 if len(pub_hex) < 70:
  y = bit.format.x_to_y(x, int(pub_hex[:2],16)%2)
 else:
  y = int(pub_hex[66:],16)
 return bytes.fromhex('04'+ hex(x)[2:].zfill(64) + hex(y)[2:].zfill(64))
def upub2cpub(upub_bytes):
    x1 = upub_bytes[1:33]
    prefix = str(2 + int(upub_bytes[33:].hex(), 16)%2).zfill(2)
    return bytes.fromhex(prefix)+x1
###############################################################################
st = time()
P = pub2upub(pubkey)
#key_seed = bytes.fromhex(hex(random.SystemRandom().randint(1,2**256))[2:])
key_seed = b''
#seed_value = b'\x01'
#random.seed(seed_value)
if out_mode == 'upub': size_key = 65
if out_mode == 'cpub': size_key = 33
if out_mode == 'xpoint': size_key = 32
if flag_binary_file:
    if out_mode == 'upub': out.write(P)             # Write Orginal Pubkey
    if out_mode == 'cpub': out.write(upub2cpub(P))        # Write Orginal Pubkey
    if out_mode == 'xpoint': out.write(P[1:33])     # Write Orginal Pubkey
else:
    if out_mode == 'upub': out.write(P.hex() + '  # Original_Pubkey\n')
    if out_mode == 'cpub': out.write(upub2cpub(P).hex() + '  # Original_Pubkey\n')
    if out_mode == 'xpoint': out.write(P[1:33].hex() + '  # Original_Pubkey\n')
res = bytearray((b'\x00') * (size_key * n))
for m in range(n):
    #key_seed = new_pos(key_seed)
    #qfix = fixrange(key_seed)
    qfix = m * 1
    tpub = bytes(bytearray(scalar_multiplication(qfix)))
    subP = bytes(bytearray(point_subtraction(P, tpub)))
    if out_mode == 'upub':  res[m*size_key:m*size_key+size_key] = bytes(subP)
    if out_mode == 'cpub':  res[m*size_key:m*size_key+size_key] = bytes(upub2cpub(subP))
    if out_mode == 'xpoint':  res[m*size_key:m*size_key+size_key] = bytes(subP[1:33])
    if m%10000==0: print('[+] Finished Total SubKeys # ', m, end= '\r')
    if not flag_binary_file:
        if out_mode == 'upub': out.write(bytes(subP).hex() + '  # -' + str(qfix)+'\n')  # Write text file line by line
        if out_mode == 'cpub': out.write(bytes(upub2cpub(subP)).hex() + '  # -' + str(qfix)+'\n')  # Write text file line by line
        if out_mode == 'xpoint': out.write(bytes(subP[1:33]).hex() + '  # -' + str(qfix)+'\n')  # Write text file line by line
if flag_binary_file:
    out.write(res)
out.flush()
os.fsync(out.fileno())
out.close()
print('[+] Finished Total SubKeys # ', n, end= '\r')
print('[-] Completed in {0:.2f} sec'.format(time() - st))

Writing PubSubs.py


In [ ]:
import bit
import hashlib, random
import platform
from time import time
import os
import sys
import ctypes
import multiprocessing

nbits = 132
low = 2**(nbits-1)
high = -1+2**nbits
diff = high - low

filename = '6000.bin'
with open(filename, 'rb') as f:
    add = f.read()#.split()7
#add = set(add)

if platform.system().lower().startswith('win'):
    dllfile = 'ice_secp256k1.dll'
    if os.path.isfile(dllfile) == True:
        pathdll = os.path.realpath(dllfile)
        ice = ctypes.CDLL(pathdll)
    else:
        print('File {} not found'.format(dllfile))

elif platform.system().lower().startswith('lin'):
    dllfile = 'ice_secp256k1.so'
    if os.path.isfile(dllfile) == True:
        pathdll = os.path.realpath(dllfile)
        ice = ctypes.CDLL(pathdll)
    else:
        print('File {} not found'.format(dllfile))
else:
    print('[-] Unsupported Platform currently for ctypes dll method. Only [Windows and Linux] is working')
    sys.exit()

ice.scalar_multiplication.argtypes = [ctypes.c_char_p, ctypes.c_char_p]            # pvk,ret
ice.point_subtraction.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]  # x1,y1,x2,y2,ret
ice.init_secp256_lib()

def scalar_multiplication(kk):
    res = (b'\x00') * 65
    pass_int_value = hex(kk)[2:].encode('utf8')
    ice.scalar_multiplication(pass_int_value, res)
    return res

def point_subtraction(pubkey1_bytes, pubkey2_bytes):
    x1 = pubkey1_bytes[1:33]
    y1 = pubkey1_bytes[33:]
    x2 = pubkey2_bytes[1:33]
    y2 = pubkey2_bytes[33:]
    res = (b'\x00') * 65
    ice.point_subtraction(x1, y1, x2, y2, res)
    return res

def new_pos(full_bytes):
    pos = hashlib.sha256(full_bytes).digest()
    return pos

def fixrange(full_bytes):
    t = low + int(full_bytes.hex(), 16) % diff
    return t

def pub2upub(pub_hex):
    x = int(pub_hex[2:66], 16)
    if len(pub_hex) < 70:
        y = bit.format.x_to_y(x, int(pub_hex[:2], 16) % 2)
    else:
        y = int(pub_hex[66:], 16)
    return bytes.fromhex('04' + hex(x)[2:].zfill(64) + hex(y)[2:].zfill(64))

def upub2cpub(upub_bytes):
    x1 = upub_bytes[1:33]
    prefix = str(2 + int(upub_bytes[33:].hex(), 16) % 2).zfill(2)
    return bytes.fromhex(prefix) + x1

def search_for_subkey(m, add, pubkey):
    m = 1 #2557120
    key_seed = b''
    while True:
        P = pub2upub(pubkey)
        qfix = m * 1329227995784915872903807060280344575
        #qfix = (random.randrange(1,10889035741470030830827987437816582766592)) # Use an interval of 1000000 for qfix
        key_seed = new_pos(key_seed)
        #qfix = fixrange(key_seed)
        tpub = bytes(bytearray(scalar_multiplication(qfix)))
        subP = bytes(bytearray(point_subtraction(P, tpub)))
        cpub = bytes(upub2cpub(subP))
        m += 1

        msg = 'Test Cpub : {total}, {num}, {password} '.format(total=m, num=qfix, password=bytes(cpub).hex())
        sys.stdout.write('\r' + msg)
        sys.stdout.flush()


        if cpub in add:
            print("Winner Found!:{num}, {password} ".format(num=qfix, password=bytes(cpub).hex()))
            f = open (u"Winner.txt","a")
            f.write("num:" + str(qfix) +'\n' +
                    "cpub:" + str(bytes(cpub).hex())+ '\n\n')
            f.close()
            break

if __name__ == "__main__":
    pubkey = "02145d2611c823a396ef6712ce0f712f09b9b4f3135e3e0aa3230fb9b6d08d1e16"
    st = time()
    processes = []

    num_processes = 2

    for i in range(num_processes):
        process = multiprocessing.Process(target=search_for_subkey, args=(i, add, pubkey))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    print('[-] Completed in {0:.2f} sec'.format(time() - st))
# 10889035741470030830827987437816582766592

Test Cpub : 2300, 3055895162309521591805852431584512177925, 027482e24cf6972f17c6cbf0cedb3c0ad9205e2d56ecbb3ef1c6e944406fabc99e 